In [1]:
!pip install cirq

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import cirq
from math import pi

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device : {device}")

Using device : cuda


In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=30, shuffle=True)

Using downloaded and verified file: ./data/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Using downloaded and verified file: ./data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 1802205.00it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 11981464.63it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=30, shuffle=False)

In [13]:
def print_custom_loading_bar(current_value, total_value, bar_length=40):
    progress = min(1.0, current_value / total_value)
    arrow = '■' * int(progress * bar_length)
    spaces = ' ' * (bar_length - len(arrow))
    print(f'\r[{arrow}{spaces}] {int(progress * 100)}%', end='', flush=True)

In [6]:
class QuantamConv2d(nn.Module):
    def __init__(self):
        super(QuantamConv2d, self).__init__()
        self.weight = nn.Parameter(torch.ones(4))*100
        self.bias = nn.Parameter(torch.zeros(4))
        self.simulator = cirq.Simulator()


    def forward(self, x):
        kernel_height, kernel_width = (2,2)
        assert x.size() == ( 30, 1, 28, 28 )
        batch, _, image_height, image_width = x.size()
        result = torch.zeros(30, 1, 14, 14)
        for z in range(30):
            for i in range(0,image_height - kernel_height + 1,2):
                for j in range(0,image_width - kernel_width + 1,2):
                    P = x[z][0][i:i+kernel_height, j:j+kernel_width]
                    P = [P[0][0], P[0][1], P[1][0], P[1][1]]
                    circuit, keys = self.kernel(P)
                    res = self.simulator.run(circuit, repetitions=10)
                    # print(res.histogram(key=keys[3]))
                    try:
                        result[z][0][i//2][j//2] = res.histogram(key=keys[3])[1] * 0.1
                    except:
                        result[z][0][i//2][j//2] = 0
        return result
    
    def backward():
        pass

    def kernel(self, P):
        Q = [cirq.GridQubit(i,0) for i in range(4)]
        W = [cirq.GridQubit(i,1) for i in range(3)]
        keys = ["q0", "q1", "q2", "q3"]

        circuit = cirq.Circuit()
        # for i in range(4):
        #     circuit.append(cirq.H(Q[i]))

        weight = self.weight.tolist()

        for i in range(4):
            circuit.append(cirq.ry(P[i].item()/255 * pi).on(Q[i]))

        for i in range(3):
            circuit.append(cirq.rx(weight[i]/255 * pi).on(W[i]))

        for i in range(3):
            circuit.append(cirq.TOFFOLI(W[i], Q[i], Q[i+1]))

        for i in range(3):
            circuit.append(cirq.ZZ(Q[i], Q[i+1]))

        for i in range(4):
            circuit.append(cirq.measure(Q[i], key=keys[i]))
        return circuit, keys



In [7]:
class CustomLayer(nn.Module):
    def __init__(self):
        super(CustomLayer, self).__init__()
        self.conv1 = QuantamConv2d()

    def forward(self, x):
        # print("before : ", x.shape)
        x = torch.relu(self.conv1(x))
        # print("after : ", x.shape)
        return x

In [38]:
# Define the CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.custom_layer = CustomLayer()
        self.fc1 = nn.Linear(14*14, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=2)

    def forward(self, x):
        # x = torch.relu(self.custom_layer(x))
        x = torch.relu(self.conv(x))
        x = x.view(-1, 1*14*14)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [49]:
# Initialize the network and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [50]:
# Training loop
total = 100
for epoch in range(total):  # Change the number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        break
    print_custom_loading_bar(epoch + 1, total)

print('Finished Training')
print("Running loss: ", running_loss)

[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100%Finished Training
Running loss:  2.284698247909546


In [51]:
# Testing the model
correct = 0
total = 0
count = 0
with torch.no_grad():
    for data in testloader:
        count += 1
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(f"total: {total}, correct = {correct}")
        if total > 1000 : break
        print_custom_loading_bar(total, 1000)
        
print(f'Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%')


total: 30, correct = 4
[■                                       ] 3%total: 60, correct = 6
[■■                                      ] 6%total: 90, correct = 16
[■■■                                     ] 9%total: 120, correct = 21
[■■■■                                    ] 12%total: 150, correct = 27
[■■■■■■                                  ] 15%total: 180, correct = 31
[■■■■■■■                                 ] 18%total: 210, correct = 32
[■■■■■■■■                                ] 21%total: 240, correct = 39
[■■■■■■■■■                               ] 24%total: 270, correct = 44
[■■■■■■■■■■                              ] 27%total: 300, correct = 46
[■■■■■■■■■■■■                            ] 30%total: 330, correct = 51
[■■■■■■■■■■■■■                           ] 33%total: 360, correct = 57
[■■■■■■■■■■■■■■                          ] 36%total: 390, correct = 60
[■■■■■■■■■■■■■■■                         ] 39%total: 420, correct = 66
[■■■■■■■■■■■■■■■■                        ] 42%total: 450, co